# Motivation

Let's recall the basic statistics we learned in the beginning.


*   Let's try to compute standard deviation for an estimator (or statistic): i.e., standard error

*   For example, in case of mean $\overline{x}$ for i.i.d. data, we know that

\begin{equation}
s^2 = \frac{\sum_{i=1}^{n} (x_i - \overline{x})^2}{n - 1}
\end{equation}

is a unbiased estimator of the standard deviation of the $\bf{data}$ (i.e., $x_i$, for $i=1,2,...,n$'s variance is estimated via $s^2$)

*   Then, the standard error of the mean is computed

\begin{equation}
V[\overline{x}] = \sqrt{\frac{s^2}{n}}
\end{equation}

which can be used to form confidence intervals or conduct hypothesis tests (in conjunction with the Central Limit Theorem).

* How do you estimate the standard error for the median of $x_1, ..., x_n$?

* What about statistical inferences about a quantile, for example the 5th percentile of $X_1, ..., X_n$? Surprisingly hard, epspecially the shape of distribution is not quite normal even after enough number of samples exist.

* Bootstrap is a computational method to construct standard error estimates of confidence interval for a wide range of estimators.


* Assume $n$ i.i.d. random variables $x_1, ..., x_n$

* Estimator of a parameter of interest such as mean, median, variance, quantile and other function of the random variable(s) $x$

* Define "Empirical Distribution Function" (EDF) $\hat{F}(X)$ as follows.

$$ \hat{F}(x) = \frac{1}{n} \sum_{i=1}^{n} I(X_i \leq x) $$

where $I(X_i \leq x)$ is the indicator function, which is 1 if $X_i \leq x$ and 0 otherwise.

The EDF is a step function that jumps up by 1/n at each of the n data points. Its value at any specified value of the measured variable is the fraction of observations of the measured variable that are less than or equal to the specified value. The EDF is an estimate of the cumulative distribution function that generated the points in the sample.

# Basic Bootstrapping (i.i.d. sampling)

1. Simulate a set of n i.i.d. uniform random integers $u_i$ , $i = 1, . . . n$ from the range $1, . . . , n$ (with replacement)

2. Construct a bootstrap sample $ x_b^⋆ = { x_{u_1},x_{u_2},...x_{u_n} }$

3. Compute the statistic of interest (e.g., mean, median, quantile, variance, regression coefficients, t-statistics, etc.) In the case of mean, compute

$$ \hat{\theta}^{*}_b = \frac{1}{n} \sum_{i=1}^n x^*_{b,i} $$

4. Repeat steps 2 and 3 for $I$ times (i.e., $I$ number of $n$ samples).

5. Compute the standard error of $\hat{\theta}$ by

$$ \frac{1}{B} \sum_{b=1}^{I} (\hat{\theta}^{*}_b - \hat{\theta})^2 $$, where $\hat{\theta}$ normally comes from your estimation but you may use an average of the bootstrapped estimates with some caution on bias (more on this).

Let's have a first example.

In [1]:
import numpy as np

n = 100
x = np.random.randn(n) # generate random numbers representing the actual samples

# Compute the mean of x
mu = np.mean(x)

# The number of bootstrapped samples
B = 1000

# Initialize muStar
muStar = np.zeros(B)

# Loop over B bootstraps
for b in range(B):
    # Uniform random numbers over 1...n
    u = np.ceil(n * np.random.rand(n)).astype(int) - 1 #random permutation of order
    # x-star sample simulation
    xStar = x[u] # Plug the random order obtained from above to the original x
    # Mean of x-star
    muStar[b] = np.mean(xStar)

s2 = 1 / (n - 1) * np.sum((x - mu) ** 2) # Original varince formula

stdErr = np.sqrt(s2 / n) #traditional standard error

bootstrapStdErr = np.sqrt(np.mean((muStar - mu) ** 2))

print("Standard Error:", stdErr)
print("Bootstrap Standard Error:", bootstrapStdErr)

Standard Error: 0.09682903961327602
Bootstrap Standard Error: 0.0963278096079583


Balanced resampling

In standard i.i.d. bootstrap, some values will inevitibly appear more than others

Balanced resampling ensures that all values appear the same number of times

In practice simple to implement.

1. Replicate the data so that there are $B$ copies of each $x_i$. The data set should have $Bn$ observations

2. Construct a random permutation of the numbers $1,...,Bn$ as $u_1,...u_{Bn} $

3. Construct the bootstrap sample $x_b^⋆ = x_{u_{n(b−1)+1}},x_{u_{n(b−1)+2}},...x_{u_{n(b−1)+n}} $

In [2]:
#Data Generation:

# Sets the number of data points in the dataset.

n = 100
# Generates n random data points from a standard normal distribution (mean = 0, standard deviation = 1).
#x = np.random.randn(n)
# Sets the number of bootstrap samples to be generated.
B = 1000

# Replicate the data
# Creates a larger array by repeating the original dataset x B times. This is done to efficiently sample bootstrap datasets later.
xRepl = np.tile(x, B)

# Random permutation of 1,...,B*n
# Generates a random permutation of integers from 0 to n*B - 1.
# This will be used to randomly select elements from the replicated dataset xRepl to form bootstrap samples.

u = np.random.permutation(n * B)

# Initialize an array to store the means of bootstrap samples
bootstrap_means = np.zeros(B)

# Loop over B bootstraps
for b in range(B):
    # Uniform random numbers over 1...n
    ind = np.arange(n * (b - 1), n * b) #Shift index
    xb = xRepl[u[ind]]
    # Compute the mean of the bootstrap sample and store it
    bootstrap_means[b] = np.mean(xb)

# Compute the standard error of the mean using the bootstrap samples
bootstrap_std_error = np.sqrt(np.var(bootstrap_means, ddof=1))

print("Bootstrap Standard Error of the Mean:", bootstrap_std_error)


Bootstrap Standard Error of the Mean: 0.09535692063701844


Antithetic Random Variables

  . If samples are negatively correlated, variance of statistics can be reduced.

  . Basic idea is to order data so that if one sample has too many large values of $x$, then the next will have too many small.

  .This can induce negative correlation while not corrupting bootstrap


In [3]:
#n = 100
#x = np.random.randn(n)


# Initialize muStar
muStar = np.zeros(B)

# Sort x
x = np.sort(x)

# Loop over B bootstraps
for b in range(0, B, 2):
    # Uniform random numbers over 1...n
    u = np.ceil(n * np.random.rand(n)).astype(int) - 1
    xStar = x[u]
    # Mean of x-star
    muStar[b] = np.mean(xStar)
    # Uniform random numbers over 1...n, mirrored
    u = n - u - 1
    xStar = x[u]
    # Mean of x-star
    if b+1 < B:  # Check to prevent index out of bounds
        muStar[b+1] = np.mean(xStar)

# Compute the correlation
correlation = np.corrcoef(muStar[0:B:2], muStar[1:B:2])[0, 1]

print("Correlation:", correlation)




Correlation: -0.9899613677885594


In [4]:
muStar = np.zeros(B)
x = np.sort(x)

for b in range(0, B, 2):
    u = np.ceil(n * np.random.rand(n)).astype(int) - 1
    xStar = x[u]
    muStar[b] = np.mean(xStar)

    u = n - u - 1
    xStar = x[u]
    if b+1 < B:
        muStar[b+1] = np.mean(xStar)

bootstrap_std_error = np.sqrt(np.var(muStar, ddof=1) / B)

print("Bootstrap Standard Error of the Mean:", bootstrap_std_error)


Bootstrap Standard Error of the Mean: 0.0029859142925219256
